<a href="https://colab.research.google.com/github/dineshjakkam/machine-learning-projects/blob/master/models/colab/Copy_of_cos_wave_model_tinyml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding the basics of TinyML
* The goal of this project is to train a simple model that can take a value x, 
and predicts its cos *(trignometric function)*, y.

* In a real-world if you need a cos value, we can just directly call it as cos(x). But as part of this learning experience, we train a model using TensorFlow lite framework to predict its cosine given a value x.
* Later will deploy this model onto the ARM based microcontroller and control one of its peripherals through the deployed model inference.

## Import libraries

In [0]:
# Install TensoFlow Library
!pip install tensorflow==2.0
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import math

## Generating samples

In [0]:
SAMPLES = 1000
# Seed value to get the same random numbers each time we run this
# notebook. This can be of any value
SEED = 1993
np.random.seed(SEED)
tf.random.set_seed(SEED)

# np.random.unifrom generates the uniformly distributed random value 
# starting 0 to 2*pi, which cover complete sine wave oscillation
x_values = np.random.uniform(low=0, high=2*math.pi, size=SAMPLES)

# Shuffle the values to make sure they are not in order
np.random.shuffle(x_values)

# calculate corresponding cos values
y_values = np.cos(x_values)

# Plot the values using matplotlib
# b dot indicates blue dots
plt.plot(x_values, y_values, 'b.')
plt.show()

### Add noise to the samples

In [0]:
# Add a small random number to each y_value
y_values += 0.1 * np.random.randn(*y_values.shape)

# Plot the values using matplotlib
# b dot indicates blue dots
plt.plot(x_values, y_values, 'b.')
plt.show()

### Split the data

In [0]:
# We use 60% of data for training, 20% for validation and the
# remianing 20% for testing. Split the indices as required
TRAIN_SPLIT = int(0.6*SAMPLES)
TEST_SPLIT = int(0.2*SAMPLES + TRAIN_SPLIT)

# Given two indices np.split chop the data into three parts
x_train, x_validate, x_test = np.split(x_values, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_validate, y_test = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

assert (x_train.size + x_validate.size + x_test.size) == SAMPLES

# Plot the data is each partition in different colors
plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_validate, y_validate, 'y.', label="Validate")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.legend()
plt.show()

## To create our model, we are going to design a simple neural newtork


1.   Create a sequential model using Keras
2.   Define three dense layers with 16 neurons in the intermediate layers



In [0]:
from tensorflow.keras import layers
model = tf.keras.Sequential()

# First layers takes a scalar input and feeds it through 16 neurons
# The *rectified linear unit* (ReLu) activation function, decide wether
# to activate the neuron or not
model.add(layers.Dense(16, activation='relu', input_shape=(1,)))

# second desned layer
model.add(layers.Dense(16, activation='relu'))

# Final layer is a single, since we want to output a single value
model.add(layers.Dense(1))

# Compile the model using standard optimizer and loss function for regression
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
model.summary()

### Train the model

In [0]:
history = model.fit(x_train, y_train, epochs=600, batch_size=16,
                    validation_data=(x_validate, y_validate))

## Graphs for the obtained results

### Plot the loss

In [0]:
# training loss
loss = history.history['loss']
# validation loss
val_loss = history.history['val_loss']
epochs = range(1, len(loss)+1)

plt.plot(epochs, loss, 'g.', label='Training Loss')
plt.plot(epochs, val_loss, 'b.', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
# To better see the result skip the first 100 epochs 
SKIP = 100
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training Loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation Loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Plot the mean absolute error

In [0]:
mae = history.history['mae']
val_mae = history.history['val_mae']

plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

## Testing


In [0]:
# Calculate and print the loss on the test dataset
loss = model.evaluate(x_test, y_test)

#Make predictions based on our test dataset
predictions = model.predict(x_test)

# Graph the predictions against the actual values
plt.clf()
plt.title('Comparison of prediction and actual values')
plt.plot(x_test, y_test, 'b.', label='Actual')
plt.plot(x_test, predictions, 'r.', label='Predictions')
plt.legend()
plt.show()

## Convert the model to the TensorFlow Lite format

### Without quantization

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save the model to disk
open("cos_model.tflite", "wb").write(tflite_model)

### With quatization

In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# indicate that we want to perform default optimization which
# includes quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# To create a quantized model that runs as efficiently as possible,
# we neeed to provide a *representative dataset* -  a set of numbers
# that represent the full range of input values of the dataset on which 
# the model was trained (as generator function)
def representative_dataset_generator():
  for value in x_test:
    # Each scalar value must be inside of a 2D array that is wrapped in a list
    yield [np.array(value, dtype=np.float32, ndmin=2)]

converter.representative_dataset = representative_dataset_generator

#convert the model
tflite_model = converter.convert()

# save the model to disk
open("cos_model_quantized.tflite", "wb").write(tflite_model)
